In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import gmaps
import os
import requests
import shutil

In [2]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [3]:
input_folder = '../data/railways/nl/shapes_voor_website/'
input_file = input_folder+'spoor.shp'

output_folder = '../data/output_images/Netherlands/'
output_csv = output_folder+'NET.csv'

update_folder = output_folder+'/set_2/'

### Import Routes

In [4]:
routes = gpd.read_file(input_file)

routes = routes.to_crs({'init' :'epsg:4269'})

routes.head()

,ID,OBJECTNAAM,GEOCODE,SPOORNUMME,KMVAN,KMTOT,geometry
0,1438188,as-spoor,950,RK,13.118,13.186,LINESTRING (4.355233512313273 51.8764085151107...
1,30843554,as-spoor,917,27,78.130,78.495,LINESTRING (4.866888980297831 52.3913574208449...
2,30843539,as-spoor,917,982AR,78.230,78.230,LINESTRING (4.868201802003703 52.3909277632807...
3,28786315,as-spoor,544,1113V,1.083,1.128,LINESTRING (4.786112083302916 51.5956615365737...
4,28826889,as-spoor,467,None,103.811,103.813,"LINESTRING (4.442601229605728 51.896203285737,..."


In [4]:
# Number of routes
routes.shape

(16544, 7)

In [6]:
tmp = routes.iloc[0]

In [15]:
list(tmp.geometry.coords)

[(4.355233512313273, 51.876408515110725),
 (4.354917511750246, 51.87638804807558),
 (4.354247125378726, 51.87634213558934)]

In [5]:
'''
Get all points
'''
# len(routes.ID.tolist())
points = []
for i in range(0,10):
    try:
        route = routes.iloc[i]
        g = list(route.geometry.coords)
        for j in range(0,len(g)):
            sec_points = g[j]
            points.append(sec_points)
    except Exception as e:
        print("Skipped route number",i,'because',e)

print(len(points),'points')

69 points


In [ ]:
'''
Get points for 1 or more routes
'''

rows = [0]

points = []
for i in range(0,len(routes.ID.tolist())):
    if i in rows:
    #     print('Route',i)
        try:
            route = routes.iloc[i]
            g = list(route.geometry.coords)
            for j in range(0,len(g)):
                sec_points = mapping(g[j])["coordinates"]
                for k in range(0,len(sec_points)):
                    sec_points = g[j]
                    points.append(sec_points)
        except Exception as e:
            print("Skipped route number",i,'because',e)

print(len(points),'points')

In [ ]:
'''
Get points for single route/section
'''

row = 0
section = 0

points = []
try:
    route = routes.iloc[row]
    g = [i for i in route.geometry]
    len(g)

    points = mapping(g[section])["coordinates"]
except Exception as e:
    print("Skipped route number",row,'because',e)
    
print(len(points),'points')

In [7]:
'''
Get subselection of points
'''

route_points = []
for point in range(0,len(points),2):
    tmp = points[point]
    route_points.append(tmp)

print(len(route_points))

35


In [8]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]

df.head()

,Name,Longitude,Latitude
0,4.355233512313273_51.876408515110725,4.355234,51.876409
1,4.354247125378726_51.87634213558934,4.354247,51.876342
2,4.867079960936101_52.39127084540921,4.867080,52.391271
3,4.868556529472031_52.39061170118105,4.868557,52.390612
4,4.869092383207482_52.390372322562406,4.869092,52.390372


In [9]:
'''
Export points
'''

df.to_csv(output_csv,header=True,index=None)

In [4]:
'''
Open csv
'''

df = pd.read_csv(output_csv)
df.head()

,Name,Longitude,Latitude,Catenary
0,4.354247125378726_51.87634213558934,4.354247,51.876342,1
1,4.355233512313273_51.876408515110725,4.355234,51.876409,1
2,4.442573076920332_51.89620581896318,4.442573,51.896206,0
3,4.514447461059589_51.89960635851382,4.514447,51.899606,1
4,4.786756389282053_51.59568526839278,4.786756,51.595685,1


In [5]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))
    
# Set map centerpoint
coords = marker_points[0]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 13
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [ ]:
'''
Get satellite preview for image
'''

row = 0

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

In [7]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+update_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 